In [3]:
import pandas as pd
import numpy as np
import lightgbm as lgb

import matplotlib.pylab as plt
%matplotlib inline
from matplotlib.pylab import rcParams

from sklearn.metrics import confusion_matrix

import pickle

import sklearn

from sklearn.metrics import classification_report

from sklearn.metrics import classification_report

import operator

In [4]:
train_set = pd.read_csv('/Users/mengruiyin/Desktop/project/xgboost/python-package/train_set.csv')

In [5]:
test_set = pd.read_csv('/Users/mengruiyin/Desktop/project/xgboost/python-package/test_set.csv')

In [6]:
train_set.set_index(['user_id', 'product_name'], inplace = True)

In [7]:
test_set.set_index(['user_id', 'product_name'], inplace = True)

In [8]:
cols = ['order_dow', 'days_since_prior_order', 'order_number', 'average_days_since_prior_order',
       'reorder_number','order_dow(mode)', 'department_id','add_to_cart_order_mean', 'recency_Mean',
       'purchase_number']
s_feature_train = train_set[cols]
y_train = train_set['label']
s_feature_test = test_set[cols]
y_test = test_set['label']

In [24]:
lgb_data_train = lgb.Dataset(s_feature_train,
                      label=y_train,
                      categorical_feature=['department_id'])

In [11]:
params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': {'binary_logloss'},
    'num_leaves': 96,
    'max_depth': 10,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.95,
    'bagging_freq': 5
}
ROUNDS = 100

### Build Model

In [9]:
lgb_model = lgb.train(params, lgb_data_train, ROUNDS)

lightgbm/basic.py:1036: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


In [12]:
y_fit_lgb = lgb_model.predict(s_feature_train)

In [13]:
y_pred_lgb = lgb_model.predict(s_feature_test)

In [14]:
train_feature_copy = s_feature_train.copy()
train_feature_copy['pred_lgb'] = y_fit_lgb

In [15]:
test_feature_copy = s_feature_test.copy()
test_feature_copy['pred_lgb'] = y_pred_lgb

In [70]:
TRESHOLD = 0.15  # guess 

train_feature_copy['fit_lgb_label'] = np.where(train_feature_copy.pred_lgb > TRESHOLD, 1, 0)

In [71]:
train_feature_copy.fit_lgb_label.values

array([0, 0, 0, ..., 0, 0, 0])

In [72]:
cfm_lgb_train = confusion_matrix(y_train.values,train_feature_copy['fit_lgb_label'].values)
print(cfm_lgb_train)

[[4572379  780368]
 [ 253067  326448]]


In [73]:
print(classification_report(y_train.values, train_feature_copy['fit_lgb_label']))

             precision    recall  f1-score   support

          0       0.95      0.85      0.90   5352747
          1       0.29      0.56      0.39    579515

avg / total       0.88      0.83      0.85   5932262



In [57]:
# TRESHOLD = 0.15  # guess, should be tuned with crossval on a subset of train data


test_feature_copy['pred_lgb_label'] = np.where(test_feature_copy.pred_lgb > TRESHOLD, 1, 0)

In [74]:
cfm_lgb = confusion_matrix(y_test.values,test_feature_copy['pred_lgb_label'].values)
print(cfm_lgb)

[[2103184  189906]
 [ 140989  108320]]


In [75]:
print(classification_report(y_test.values, test_feature_copy['pred_lgb_label']))

             precision    recall  f1-score   support

          0       0.94      0.92      0.93   2293090
          1       0.36      0.43      0.40    249309

avg / total       0.88      0.87      0.87   2542399



In [76]:
sklearn.metrics.f1_score(y_test.values, test_feature_copy['pred_lgb_label'])

0.39566420411480546

### Cross Validation

Tune Parameters for the Leaf-wise Tree
* num_leaves: control the complexity of the tree model. Theoretically, we can set num_leaves = 2^(max_depth). To avoid overfitting, we usually set the parameter smaller than 2^(max_depth).
* max_depth: control the tree depth explicitly

For Faster Speed:
* bagging_fraction: use bagging
* bagging_freq: use bagging
* feature_fraction: use feature sub-sampling


For Better Accuray:
* learning_rate
* num_leaves

#### Tune parameters for lgb model

In [26]:
learning_rate = [0.01, 0.05, 0.1]
max_depth=[6,8,10,12]
num_leaves=[128,96,192,256]
feature_fraction = [0.7,0.8,0.9]
bagging_fraction = [0.7,0.8,0.9,0.95]
bagging_freq =  5
results= {}
for md in max_depth:
    params['max_depth']=md
    for nl in num_leaves:
        params['num_leaves']=nl
        for ff in feature_fraction:
            params['feature_fraction']=ff
            for ll in learning_rate:
                params['learning_rate']=ll
                for bf in bagging_fraction:
                    params['bagging_fraction']=bf
                    param=[md,nl,ff,bf, ll]
                    print param
                    lgb_data_train = lgb.Dataset(s_feature_train,label=y_train,
                      categorical_feature=['department_id'])
                    bst=lgb.train(params,lgb_data_train, ROUNDS)
                    fit = bst.predict(s_feature_train)
                    train_feature_copy['tune_fit'] = fit
                    train_feature_copy['tune_label'] = np.where(train_feature_copy.tune_fit > 0.21, 1,0)
                    f1_score = sklearn.metrics.f1_score(y_train.values, train_feature_copy['tune_label'])
                    results[repr(param)] = f1_score
            
f = open('gridsearch_result.save', 'wb')
cPickle.dump(results, f, protocol=cPickle.HIGHEST_PROTOCOL)
f.close()

[6, 128, 0.7, 0.7, 0.01]
[6, 128, 0.7, 0.8, 0.01]
[6, 128, 0.7, 0.9, 0.01]
[6, 128, 0.7, 0.95, 0.01]
[6, 128, 0.7, 0.7, 0.05]
[6, 128, 0.7, 0.8, 0.05]
[6, 128, 0.7, 0.9, 0.05]
[6, 128, 0.7, 0.95, 0.05]
[6, 128, 0.7, 0.7, 0.1]
[6, 128, 0.7, 0.8, 0.1]
[6, 128, 0.7, 0.9, 0.1]
[6, 128, 0.7, 0.95, 0.1]
[6, 128, 0.8, 0.7, 0.01]
[6, 128, 0.8, 0.8, 0.01]
[6, 128, 0.8, 0.9, 0.01]
[6, 128, 0.8, 0.95, 0.01]
[6, 128, 0.8, 0.7, 0.05]
[6, 128, 0.8, 0.8, 0.05]
[6, 128, 0.8, 0.9, 0.05]
[6, 128, 0.8, 0.95, 0.05]
[6, 128, 0.8, 0.7, 0.1]
[6, 128, 0.8, 0.8, 0.1]
[6, 128, 0.8, 0.9, 0.1]
[6, 128, 0.8, 0.95, 0.1]
[6, 128, 0.9, 0.7, 0.01]
[6, 128, 0.9, 0.8, 0.01]
[6, 128, 0.9, 0.9, 0.01]
[6, 128, 0.9, 0.95, 0.01]
[6, 128, 0.9, 0.7, 0.05]
[6, 128, 0.9, 0.8, 0.05]
[6, 128, 0.9, 0.9, 0.05]
[6, 128, 0.9, 0.95, 0.05]
[6, 128, 0.9, 0.7, 0.1]
[6, 128, 0.9, 0.8, 0.1]
[6, 128, 0.9, 0.9, 0.1]
[6, 128, 0.9, 0.95, 0.1]
[6, 96, 0.7, 0.7, 0.01]
[6, 96, 0.7, 0.8, 0.01]
[6, 96, 0.7, 0.9, 0.01]
[6, 96, 0.7, 0.95, 0.01]
[6, 96

[8, 256, 0.7, 0.7, 0.1]
[8, 256, 0.7, 0.8, 0.1]
[8, 256, 0.7, 0.9, 0.1]
[8, 256, 0.7, 0.95, 0.1]
[8, 256, 0.8, 0.7, 0.01]
[8, 256, 0.8, 0.8, 0.01]
[8, 256, 0.8, 0.9, 0.01]
[8, 256, 0.8, 0.95, 0.01]
[8, 256, 0.8, 0.7, 0.05]
[8, 256, 0.8, 0.8, 0.05]
[8, 256, 0.8, 0.9, 0.05]
[8, 256, 0.8, 0.95, 0.05]
[8, 256, 0.8, 0.7, 0.1]
[8, 256, 0.8, 0.8, 0.1]
[8, 256, 0.8, 0.9, 0.1]
[8, 256, 0.8, 0.95, 0.1]
[8, 256, 0.9, 0.7, 0.01]
[8, 256, 0.9, 0.8, 0.01]
[8, 256, 0.9, 0.9, 0.01]
[8, 256, 0.9, 0.95, 0.01]
[8, 256, 0.9, 0.7, 0.05]
[8, 256, 0.9, 0.8, 0.05]
[8, 256, 0.9, 0.9, 0.05]
[8, 256, 0.9, 0.95, 0.05]
[8, 256, 0.9, 0.7, 0.1]
[8, 256, 0.9, 0.8, 0.1]
[8, 256, 0.9, 0.9, 0.1]
[8, 256, 0.9, 0.95, 0.1]
[10, 128, 0.7, 0.7, 0.01]
[10, 128, 0.7, 0.8, 0.01]
[10, 128, 0.7, 0.9, 0.01]
[10, 128, 0.7, 0.95, 0.01]
[10, 128, 0.7, 0.7, 0.05]
[10, 128, 0.7, 0.8, 0.05]
[10, 128, 0.7, 0.9, 0.05]
[10, 128, 0.7, 0.95, 0.05]
[10, 128, 0.7, 0.7, 0.1]
[10, 128, 0.7, 0.8, 0.1]
[10, 128, 0.7, 0.9, 0.1]
[10, 128, 0.7, 0.95,

[12, 192, 0.7, 0.8, 0.05]
[12, 192, 0.7, 0.9, 0.05]
[12, 192, 0.7, 0.95, 0.05]
[12, 192, 0.7, 0.7, 0.1]
[12, 192, 0.7, 0.8, 0.1]
[12, 192, 0.7, 0.9, 0.1]
[12, 192, 0.7, 0.95, 0.1]
[12, 192, 0.8, 0.7, 0.01]
[12, 192, 0.8, 0.8, 0.01]
[12, 192, 0.8, 0.9, 0.01]
[12, 192, 0.8, 0.95, 0.01]
[12, 192, 0.8, 0.7, 0.05]
[12, 192, 0.8, 0.8, 0.05]
[12, 192, 0.8, 0.9, 0.05]
[12, 192, 0.8, 0.95, 0.05]
[12, 192, 0.8, 0.7, 0.1]
[12, 192, 0.8, 0.8, 0.1]
[12, 192, 0.8, 0.9, 0.1]
[12, 192, 0.8, 0.95, 0.1]
[12, 192, 0.9, 0.7, 0.01]
[12, 192, 0.9, 0.8, 0.01]
[12, 192, 0.9, 0.9, 0.01]
[12, 192, 0.9, 0.95, 0.01]
[12, 192, 0.9, 0.7, 0.05]
[12, 192, 0.9, 0.8, 0.05]
[12, 192, 0.9, 0.9, 0.05]
[12, 192, 0.9, 0.95, 0.05]
[12, 192, 0.9, 0.7, 0.1]
[12, 192, 0.9, 0.8, 0.1]
[12, 192, 0.9, 0.9, 0.1]
[12, 192, 0.9, 0.95, 0.1]
[12, 256, 0.7, 0.7, 0.01]
[12, 256, 0.7, 0.8, 0.01]
[12, 256, 0.7, 0.9, 0.01]
[12, 256, 0.7, 0.95, 0.01]
[12, 256, 0.7, 0.7, 0.05]
[12, 256, 0.7, 0.8, 0.05]
[12, 256, 0.7, 0.9, 0.05]
[12, 256, 0.7, 

NameError: name 'cPickle' is not defined

In [33]:
max_key = max(results.iteritems(), key=operator.itemgetter(1))[0]
max_key

'[12, 256, 0.7, 0.8, 0.1]'

In [32]:
results[max_key]

0.4009910774791055

In [9]:
params_result = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': {'binary_logloss'},
    'num_leaves': 256,
    'max_depth': 12,
    'feature_fraction': 0.7,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'learning_rate': 0.1
}
ROUNDS = 100

#### Fit the model with best parameters

In [10]:
lgb_data_train = lgb.Dataset(s_feature_train,
                      label=y_train,
                      categorical_feature=['department_id'])

In [38]:
bst = lgb.train(params_result, lgb_data_train, ROUNDS)

In [55]:
y_fit_bst = bst.predict(s_feature_train)

In [57]:
train_set_copy = train_set.copy()
train_set_copy['fit_bst'] = y_fit_bst

#### Tune threshold

In [58]:
TRESHOLD = [0.15,0.16,0.17,0.18,0.19,0.2,0.21,0.22,0.23,0.24,0.25]  # guess, should be tuned with crossval on a subset of train data

In [60]:
thre = {}
for t in TRESHOLD:
    train_set_copy['bst_label'] = np.where(train_set_copy.fit_bst > t, 1,0)
    f1_score = sklearn.metrics.f1_score(y_train.values, train_set_copy['bst_label'])
    thre[t] = f1_score

In [62]:
max_key2 = max(thre.iteritems(), key=operator.itemgetter(1))[0]
max_key2

0.2

In [40]:
y_pred_bst = bst.predict(s_feature_test)

In [63]:
test_set_copy = test_set.copy()
test_set_copy['pred_bst'] = y_pred_bst

In [64]:
test_set_copy['bst_label'] = np.where(test_set_copy.pred_bst > max_key2, 1,0)

In [65]:
cfm_bst = confusion_matrix(y_test.values,test_set_copy['bst_label'].values)
print(cfm_bst)

[[2086461  206629]
 [ 136492  112817]]


In [66]:
print(classification_report(y_test.values, test_set_copy['bst_label']))

             precision    recall  f1-score   support

          0       0.94      0.91      0.92   2293090
          1       0.35      0.45      0.40    249309

avg / total       0.88      0.87      0.87   2542399



The f1-score for the test data of lgb is 0.4, much better than logistic regression and xgboost. So I would choose gradient boosting as the final model. 